#### This notebook reads ``uk_addresses.csv`` and calls ``api.openalpr.com`` to get car metdata like model, colour, type etc. The output file it creates is a JSON file that then gets ingested into Postgresql Database

In [1]:
#!/usr/bin/python

import requests
import base64
import json
import uuid
#pip3 install Faker
from faker import Faker
import random
import pandas
from geopy.geocoders import Nominatim

def seed_db_creator(IMAGE_PATH):
  SECRET_KEY = 'sk_90c281aebbfc04d52d6c2e86 0KR'
  results= {}
  with open(IMAGE_PATH, 'rb') as image_file:
      img_base64 = base64.b64encode(image_file.read())
  url = 'https://api.openalpr.com/v3/recognize_bytes?recognize_vehicle=1&country=us&secret_key=%s' % (SECRET_KEY)
  r = requests.post(url, data = img_base64)
  data = json.loads( json.dumps(r.json(), indent=2))
  
  ## Uncomment if you want to get input data from json instead of openalpr.com API
  dump_file = open("database_api_response.json")
  data = json.load(dump_file)

 ## Read UK Addresses CSV File
  df = pandas.read_csv("uk_addresses.csv")
  #address_file = open("addresses.json")
  #address_data = json.load(address_file)
  
  for i in data["results"]:
    customer_id = str(uuid.uuid4())
    fake_owner_name = fake.name()
    random_int = random.randint(0,49)

    #print(fake_address_details)
    #fake_lat_long = fake.local_latlng()

    ##### Vehicle Information #####
    results.update({"vehicle_registered_plate_number": i['plate']})
    results.update({"vehicle_color": i["vehicle"]["color"][0]["name"]})
    results.update({"vehicle_make": i["vehicle"]["make"][0]["name"]})
    results.update({"vehicle_body_type": i["vehicle"]["body_type"][0]["name"]})
    results.update({"vehicle_make_model": i["vehicle"]["make_model"][0]["name"]})
    results.update({"vehicle_model_year": i["vehicle"]["year"][0]["name"]})
    results.update({"vehicle_registered_city": df.loc[random_int].at["City"]})
   ###### Owner Information #####
    results.update({"vehicle_owner_name": fake_owner_name})
    results.update({"vehicle_owner_address": df.loc[random_int].at["Address"]})
    results.update({"vehicle_owner_city": df.loc[random_int].at["City"]})
    results.update({"vehicle_owner_zip_code": df.loc[random_int].at["Postcode"]})
    results.update({"vehicle_owner_contact_number": df.loc[random_int].at["Phone number"]})
   ###### Customer Information #####
    results.update({"customer_id": customer_id})
    results.update({"customer_balance": str(random.randint(100,999)) + " GBP"})
    results.update({"customer_name": fake_owner_name})
    results.update({"customer_address": df.loc[random_int].at["Address"]})
    results.update({"customer_city": df.loc[random_int].at["City"]})
    results.update({"customer_zip_code": df.loc[random_int].at["Postcode"]})
    results.update({"customer_contact_number": df.loc[random_int].at["Phone number"]})
    temp = final_results['seed_database']
    temp.append(results)
    with open('database.json', 'w', encoding='utf-8') as f:
      json.dump(temp, f, ensure_ascii=False, indent=4)
    print(temp)  

final_results = {"seed_database": [ ] }
fake = Faker( 'en_GB')
#print(fake.locales)

for i in range(11,12):
  IMAGE_PATH = '/Users/karasing/git/jumpstart-library/pattern2-licence-plates/LPR_Service/dataset/images/'+str(i)+'.png'
  #print(IMAGE_PATH)
  seed_db_creator(IMAGE_PATH)


[{'vehicle_registered_plate_number': 'CHIAN000', 'vehicle_color': 'black', 'vehicle_make': 'toyota', 'vehicle_body_type': 'sedan-wagon', 'vehicle_make_model': 'toyota_land-cruiser', 'vehicle_model_year': '2010-2014', 'vehicle_registered_city': 'London', 'vehicle_owner_name': 'Dr. Ben Ingram', 'vehicle_owner_address': 'Flat 2 184 Portobello Road', 'vehicle_owner_city': 'London', 'vehicle_owner_zip_code': 'W11 2EB', 'vehicle_owner_contact_number': '+44202802 2386', 'customer_id': '16c4980f-6432-457c-baaa-dea137dfdf4a', 'customer_balance': '820 GBP', 'customer_name': 'Dr. Ben Ingram', 'customer_address': 'Flat 2 184 Portobello Road', 'customer_city': 'London', 'customer_zip_code': 'W11 2EB', 'customer_contact_number': '+44202802 2386'}]


### Reading resultant json file and ingesting the data to Postgresql Database

In [3]:
! pip3 install sqlalchemy psycopg2

  Using cached psycopg2-2.8.6.tar.gz (383 kB)
  Created wheel for psycopg2: filename=psycopg2-2.8.6-cp39-cp39-macosx_11_0_x86_64.whl size=136081 sha256=8ede2815a52f1248955b26d91b6d9cc27edee7b58375c78b9b822a79af958c56
  Stored in directory: /Users/karasing/Library/Caches/pip/wheels/a2/07/10/a9a82e72d50feb8d646acde6a88000bbf2ca0f82e41aea438a
Successfully built psycopg2


In [24]:
from sqlalchemy import create_engine, Column, Integer, String, Sequence
from sqlalchemy.ext.declarative import declarative_base
import json, urllib.request

## JSON Data URL
JSON_DB_URL = "https://raw.githubusercontent.com/red-hat-data-services/jumpstart-library/ksingh-tf-v1/pattern2-licence-plates/LPR_seed_database/uk_database.json"

## Database details and connection
DB_USER = os.getenv('DB_USER', 'dbadmin')
DB_PASSWORD = os.getenv('DB_PASSWORD', 'HT@1202k')
DB_HOST = os.getenv('DB_HOST', '127.0.0.1')
DB_NAME = os.getenv('DB_NAME','pgdb')
TABLE_NAME = os.getenv('TABLE_NAME','vehicle_metadata')

engine = create_engine('postgresql://'+DB_USER+':'+DB_PASSWORD+'@'+DB_HOST+'/'+DB_NAME, connect_args={})
Base = declarative_base()
TABLE_ID = Sequence(TABLE_NAME+'_id_seq', start=1000)

class Create_Table(Base):
    __tablename__ = TABLE_NAME
    id = Column(Integer, TABLE_ID,  primary_key=True, index=True, server_default=TABLE_ID.next_value())
    vehicle_registered_plate_number = Column(String, primary_key=True, index=True, unique=True)
    vehicle_color = Column(String)
    vehicle_make = Column(String)
    vehicle_body_type = Column(String)
    vehicle_make_model = Column(String)
    vehicle_model_year = Column(String)
    vehicle_registered_city = Column(String)
    vehicle_owner_name = Column(String)
    vehicle_owner_address = Column(String)
    vehicle_owner_city = Column(String)
    vehicle_owner_zip_code = Column(String)
    vehicle_owner_contact_number = Column(String)
    customer_id = Column(String)
    customer_balance = Column(String)
    customer_name = Column(String)
    customer_address = Column(String)
    customer_city = Column(String)
    customer_zip_code = Column(String)
    customer_contact_number = Column(String)

## Create Table if does not exists
Create_Table.__table__.create(bind=engine, checkfirst=True)

with urllib.request.urlopen(JSON_DB_URL) as url:
    data = json.loads(url.read().decode())

connection = engine.connect()

for count in range(len(data)):
    try:
        connection.execute(f"""INSERT INTO public.{TABLE_NAME}(vehicle_registered_plate_number,vehicle_color,vehicle_make,vehicle_body_type,vehicle_make_model,vehicle_model_year,vehicle_registered_city,vehicle_owner_name,vehicle_owner_address,vehicle_owner_city,vehicle_owner_zip_code,vehicle_owner_contact_number,customer_id,customer_balance,customer_name,customer_address,customer_city,customer_zip_code,customer_contact_number) 
VALUES('{data[count]['vehicle_registered_plate_number']}', '{data[count]['vehicle_color']}', '{data[count]['vehicle_make']}', '{data[count]['vehicle_body_type']}', '{data[count]['vehicle_make_model']}', '{data[count]['vehicle_model_year']}', '{data[count]['vehicle_registered_city']}', '{data[count]['vehicle_owner_name']}', '{data[count]['vehicle_owner_address']}', '{data[count]['vehicle_owner_city']}', '{data[count]['vehicle_owner_zip_code']}', '{data[count]['vehicle_owner_contact_number']}', '{data[count]['customer_id']}', '{data[count]['customer_balance']}', '{data[count]['customer_name']}', '{data[count]['customer_address']}', '{data[count]['customer_city']}', '{data[count]['customer_zip_code']}', '{data[count]['customer_contact_number']}')""")
    except:
        print("====================== Error inserting record to database ======================")
        print(sys.exc_info()[1])
    else:
        print(" Record added to DB successfully")


All Record added to DB successfully
All Record added to DB successfully
All Record added to DB successfully
All Record added to DB successfully
All Record added to DB successfully
All Record added to DB successfully
All Record added to DB successfully
All Record added to DB successfully
All Record added to DB successfully
All Record added to DB successfully
All Record added to DB successfully
All Record added to DB successfully
All Record added to DB successfully
All Record added to DB successfully
All Record added to DB successfully
All Record added to DB successfully
All Record added to DB successfully
